In [3]:
#!pip install pyatmo

In [4]:
# Import des packages
import pyatmo
from datetime import datetime as dt
import datetime


In [5]:
# Import du package qui doit préalablement être installé


In [6]:
# Information sur l'application qui est à créer depuis https://dev.netatmo.com/
CLIENT_ID = str('606609b7b70a421ea72b545a')
CLIENT_SECRET = 'XPgT5THYut4ska9BaRtsf3MeXZ6jIUz1yOeoKZ'
# Mail et mot de passe renseignés lors de la connexion du capteur
USERNAME = 'marie-lise.pannier@univ-angers.fr'
PASSWORD = 'mdp-ProjetPEIP2'
# Adresse MAC du capteur disponible depuis les paramètres avancés de l'application Smartphone Home Coach
DEVICE = '70:ee:50:7a:5f:10' 

In [7]:
# Fonction pour l'identification
authorization = pyatmo.ClientAuth(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    username=USERNAME,
    password=PASSWORD,
    scope="read_homecoach",
)

In [8]:
# Demande d'accès aux données du capteur
homeData = pyatmo.HomeCoachData(authorization)

In [9]:
# Info sur la station
homeData.stations[DEVICE]

{}

In [10]:
# Température et humidité min et max mesurées
homeData.get_min_max_t_h(DEVICE)

(14.9, 17, 48, 56)

In [11]:
# Types de données enregistrées
homeData.get_monitored_conditions(DEVICE)

[]

In [13]:
# Récupération des données sur le temperature, CO2,  à un pas de temps de 5 min
# entrée de la fonction get_data : 'device_id', 'scale', and 'module_type'
pas=1616696550
temperature=homeData.get_data(DEVICE,'5min','temperature',date_begin=str(pas),date_end=str(pas+250000))
co2=homeData.get_data(DEVICE,'5min','co2',date_begin=str(pas),date_end=str(pas+250000))
humidity=homeData.get_data(DEVICE,'5min','humidity',date_begin=str(pas),date_end=str(pas+250000))
noise=homeData.get_data(DEVICE,'5min','noise',date_begin=str(pas),date_end=str(pas+250000))
Pressure=homeData.get_data(DEVICE,'5min','Pressure',date_begin=str(pas),date_end=str(pas+250000))


ts = datetime.datetime.now().timestamp() 

# chargement de toutes les données dans la base
while pas<ts:
    pas=pas+250000
    temperature_temp=homeData.get_data(DEVICE,'5min','temperature',date_begin=str(pas),date_end=str(pas+250000))
    co2_temp=homeData.get_data(DEVICE,'5min','co2',date_begin=str(pas),date_end=str(pas+250000))
    humidity_temp=homeData.get_data(DEVICE,'5min','humidity',date_begin=str(pas),date_end=str(pas+250000))
    noise_temp=homeData.get_data(DEVICE,'5min','noise',date_begin=str(pas),date_end=str(pas+250000))
    Pressure_temp=homeData.get_data(DEVICE,'5min','Pressure',date_begin=str(pas),date_end=str(pas+250000))
    
    temperature['body'].update(temperature_temp['body'])
    co2['body'].update(co2_temp['body'])
    humidity['body'].update(humidity_temp['body'])
    noise['body'].update(noise_temp['body'])
    Pressure['body'].update(Pressure_temp['body'])
    

ApiError: 403 - Forbidden - User usage reached (26) when accessing 'https://api.netatmo.com/api/getmeasure'

In [10]:
# NB : première mesure à ne pas prendre en compte (réalisée avant calibrage)
# à faire :
# -> traduire les dates en un format compréhensible
# -> enregistrer le tout en csv par expl

In [21]:
import csv
delimiter=';'
with open('DonnesNetatmo1mai.csv', '+w', encoding='utf-8') as stream:
        writer = csv.writer(stream, delimiter=delimiter, quoting=csv.QUOTE_NONE, quotechar='',  lineterminator='\n')
        writer.writerow(['time' , 'temperature','co2','humidity','noise','Pressure'])
        for t,c,h,n,p in zip(temperature['body'].items(),co2['body'].items(),humidity['body'].items(),noise['body'].items(),Pressure['body'].items()):
            writer.writerow([int(t[0]) , t[1][0],c[1][0],h[1][0],n[1][0],p[1][0]])
